In [1]:
# Imports
import rasterio
from rasterio.windows import Window
import time
from skimage.io import imread
from PIL import Image
import numpy as np
import os
import h5py
import cv2

In [2]:
# GeoTIFF from NASA src path

topoPathA1 = 'data/topography_10800x10800_A1.tif'
#topoPathA2 = 'data/topography_10800x10800_A2.tif'
topoPathB1 = 'data/topography_10800x10800_B1.tif'
topoPathB2 = 'data/topography_10800x10800_B2.tif'
topoPathC1 = 'data/topography_10800x10800_C1.tif'
topoPathC2 = 'data/topography_10800x10800_C2.tif'
topoPathD1 = 'data/topography_10800x10800_D1.tif'
topoPathD2 = 'data/topography_10800x10800_D2.tif'

topoArrayPath = []
topoArrayPath.append(topoPathA1)
#topoArrayPath.append(topoPathA2)
topoArrayPath.append(topoPathB1)
topoArrayPath.append(topoPathB2)
topoArrayPath.append(topoPathC1)
topoArrayPath.append(topoPathC2)
topoArrayPath.append(topoPathD1)
topoArrayPath.append(topoPathD2)

wordPathA1 = 'data/word_21600x21600_A1.png'
#wordPathA2 = 'data/word_21600x21600_A2.png'
wordPathB1 = 'data/word_21600x21600_B1.png'
wordPathB2 = 'data/word_21600x21600_B2.png'
wordPathC1 = 'data/word_21600x21600_C1.png'
wordPathC2 = 'data/word_21600x21600_C2.png'
wordPathD1 = 'data/word_21600x21600_D1.png'
wordPathD2 = 'data/word_21600x21600_D2.png'

wordArrayPath = []
wordArrayPath.append(wordPathA1)
#wordArrayPath.append(wordPathA2)
wordArrayPath.append(wordPathB1)
wordArrayPath.append(wordPathB2)
wordArrayPath.append(wordPathC1)
wordArrayPath.append(wordPathC2)
wordArrayPath.append(wordPathD1)
wordArrayPath.append(wordPathD2)

# Size of tiles
xSize, ySize = 256, 256


In [3]:
# returns True if we should keep map tile, otherwise False to discard it
def comparator(heightmap_chunk):
    
    #if heightmap_chunk.mean() < 6:
     #   return False
    
    black_ratio = ((heightmap_chunk <= 45).sum()*1.0)/float(np.prod(heightmap_chunk.shape))
    #white_ratio = ((word_chunk >= 200).sum())/float(np.prod(word_chunk.shape))

    if black_ratio > 0.7:
        return False
    return True

In [4]:
# Cut geotiff images and save to folder, remove images where 75% area is water
iterator = 0
tileIterator = 0

if not os.path.exists('heightmaps'):
    os.makedirs('heightmaps')
    
#if not os.path.exists('textures512'):
#    os.makedirs('textures512')

Image.MAX_IMAGE_PIXELS = None
    
for srcTifPath in topoArrayPath:
    t0 = time.time()
    insideIterator = 0
    
    #wordTexture = imread(wordArrayPath[tileIterator])
    #wordTexture = cv2.pyrDown(wordTexture, dstsize=(10800, 10800))
        
    tileIterator += 1
    with rasterio.open(srcTifPath) as src:  
        for y in range(0, src.width, 80):
            for x in range(0, src.height, 80):
                
                
                #tile B1 - skip snow areas
                if(y < 5400 and tileIterator == 2):
                    continue
                
                #tile B2 C2 D2 - skip snow areas
                if(y > 5400 and (tileIterator == 3 or tileIterator == 5 or tileIterator == 7)):
                    continue
                              
                cropTextureX = x+xSize
                cropTextureY = y+ySize
                
                if cropTextureX > 10800 or cropTextureY > 10800:
                    continue
                
                iterator += 1
                insideIterator += 1
                # Create a Window and calculate the transform from the source dataset    
                window = Window(x, y, xSize, ySize)
                transform = src.window_transform(window)

                # Create a new cropped raster to write to
                profile = src.profile
                profile.update({
                    'height': xSize,
                    'width': ySize,
                    'transform': transform})
            
                savePathName = 'heightmaps/topography_' + str(iterator) + '.tif'
                with rasterio.open(savePathName, 'w', **profile) as dst:
                    # Read the data from the window and write it to the output raster
                    dst.write(src.read(window=window))
                    #topographyList.append(src.read(window=window))
            
                textureHeight = imread(savePathName)
            #    wordChunk = wordTexture[y:y+ySize, x:x+xSize]
            
                if not comparator(textureHeight):
                    os.remove(savePathName)
                    iterator -= 1
                    insideIterator -= 1
           #     else:
           #         wordImage = Image.fromarray(wordChunk)
           #         wordImage.save('textures512/word_' + str(iterator) + '.png')
                    
                    
    print(str(time.time()-t0) + 's for ' + srcTifPath + '. Saved ' + str(insideIterator) + ' images')

83.55174350738525s for data/topography_10800x10800_A1.tif. Saved 1291 images
42.24352550506592s for data/topography_10800x10800_B1.tif. Saved 90 images
45.452723026275635s for data/topography_10800x10800_B2.tif. Saved 416 images
92.23074293136597s for data/topography_10800x10800_C1.tif. Saved 1515 images
46.085394859313965s for data/topography_10800x10800_C2.tif. Saved 733 images
97.22769951820374s for data/topography_10800x10800_D1.tif. Saved 1506 images
44.891459941864014s for data/topography_10800x10800_D2.tif. Saved 35 images
